In [1]:
import ee
import geopandas as gpd
import pandas as pd
import os
import datetime

In [ ]:
# # Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [5]:
# read inputs from spreadsheet
working_dr = os.getcwd()
inputs_dr = os.path.join(working_dr, 'inputs.csv')
inputs_df = pd.read_csv(inputs_dr)

In [9]:
working_dr

'c:\\Users\\Nawaf\\Desktop\\mohammed\\\u200f\u200fgeeSEBAL_final'

In [6]:
# choose a season

# by index
index = 0
row = inputs_df.iloc[index]
# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]

season = row['season']
project_shp_location = row['project_shp']
field_shp_location = row['field_shp']
start_date = row['start_date']
end_date = row['end_date']
kc_df = pd.read_csv(os.path.join(working_dr,row['kc_dr']))

In [8]:
project_shp_location

'shapefile\\eg_folder\\eg_project.shp'

In [ ]:
# make dataframe for kc and dates for the specified season

# get the percentage of every duration
durations_sum = kc_df['duration'].sum()
durations_percentage = []
for i in kc_df['duration']:
    durations_percentage.append(i/durations_sum)

s_d = datetime.datetime.strptime(start_date,'%Y-%m-%d')
e_d = datetime.datetime.strptime(end_date,'%Y-%m-%d')

kc_dates = [(s_d + datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range((e_d-s_d).days + 1)]

# get the number of days for every duration
days_num = len(kc_dates)
new_durations = []
for dur_per in durations_percentage:
    new_durations.append(round(dur_per * days_num+0.05))

kc_list = []
for i in range(len(kc_df)):
    for d in range(new_durations[i]):
        kc_list.append(kc_df['kc'][i])

# ensure lists are the same size
if len(kc_dates) < len(kc_list):
    kc_list = kc_list[:len(kc_dates)]

if len(kc_list) < len(kc_dates):
    kc_dates = kc_dates[:len(kc_list)]

kc_dates_dic = {'date':kc_dates ,'kc': kc_list}
kc_dates_df = pd.DataFrame(kc_dates_dic)

In [7]:
# convert shapefile to featurecollection

project_shp = gpd.read_file(project_shp_location) #read shapefile
field_shp = gpd.read_file(field_shp_location) #read shapefile

# get area of interest (shapefile bbox)
aoi = ee.Geometry.Rectangle(project_shp.total_bounds.tolist())

# get field area
geojson = field_shp.__geo_interface__
aoi_field = ee.FeatureCollection(geojson['features'])

DriverError: shapefile\eg_folder\eg_project.shp: No such file or directory

In [ ]:
# landsat images
landsat_imgs = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filterBounds(aoi)\
  .filter(ee.Filter.date(start_date, end_date))

In [ ]:
# get landsat images as ordered list
list_size = landsat_imgs.size().getInfo()
ls8_list = landsat_imgs.toList(list_size)

In [ ]:
#Import geeSEBAL Image function
from geesebal import Image

In [ ]:
#Using Image function to estimate Daily ET with geeSEBAL
result_imgs = []
for i in range(list_size):
    img = ee.Image(ls8_list.get(i))
    landsat_img = Image(img.get('system:id').getInfo(), kc_dates_df)
    result_imgs.append(landsat_img)
    print(f"{i+1} / {list_size} done")
print('all done')

# export

In [ ]:
from geesebal.export import export_to_drive, export_png_images, write_reducers_to_excel, images_plt_layout, image_combination, save_plot_maxmeanmin, save_plot_et_etp, shapefile_image, export_priefs

In [ ]:
# create export folders
current_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
shp_name = os.path.basename(project_shp_location).split('.')[0]
project_dr = os.path.join(working_dr, 'data', shp_name, current_time)
season_dr = os.path.join(working_dr, 'data', shp_name, current_time, f'season_{season}')

et_folder = os.path.join(season_dr, 'ET')

ndvi_folder = os.path.join(season_dr, 'ndvi')

rgb_folder = os.path.join(season_dr, 'rgb')

In [ ]:
# dataframe to save statistics in csv files
sheet_data = {'date':[], 'max':[], 'mean':[], 'stdDev':[], 'min':[]}
eta_sheet_df = pd.DataFrame(sheet_data)
etp_sheet_df = pd.DataFrame({'date':[], 'mean':[], 'kc':[], 'kc*ETp':[]})
ndvi_sheet_df = pd.DataFrame(sheet_data)

In [ ]:
# # export

for img in result_imgs:
    # get pic date
    date = img.date_string

    # export tif (to drive)
    # export_to_drive(img, date, exp_folder_id, aoi)
    
    # export png images
    export_png_images(img, 'RGB', 'RGB', rgb_folder, date, aoi)
    export_png_images(img, 'ET_24h', 'ETa', os.path.join(et_folder, 'ETa'), date, aoi)
    export_png_images(img, 'ET_ETp', 'ETa_ETp', os.path.join(et_folder, 'ETa_ETp'), date, aoi)
    export_png_images(img, 'NDVI', 'NDVI', ndvi_folder, date, aoi)

    # write on spreadsheets
    write_reducers_to_excel(img, eta_sheet_df, etp_sheet_df, ndvi_sheet_df, kc_dates_df, date, aoi_field)

# export layouts  
images_plt_layout(os.path.join(et_folder, 'ETa'), 'ETa')
images_plt_layout(os.path.join(et_folder, 'ETa_ETp'), 'ETa_ETp')
images_plt_layout(ndvi_folder, 'NDVI')

# export combinations
image_combination(os.path.join(et_folder, 'ETa', 'images', 'layout'), 'ETa')
image_combination(os.path.join(rgb_folder, 'images', 'raw'), 'RGB')
image_combination(os.path.join(et_folder, 'ETa_ETp', 'images', 'layout'), 'ETa_ETp')
image_combination(os.path.join(ndvi_folder, 'images', 'layout'), 'NDVI')

# save spreadsheet to csv
eta_sheet_dr = os.path.join(et_folder, 'ETa', 'sheets')
etp_sheet_dr = os.path.join(et_folder, 'ETa_ETp', 'sheets')
ndvi_sheet_dr = os.path.join(ndvi_folder, 'sheets')
if not os.path.isdir(eta_sheet_dr):
    os.makedirs(eta_sheet_dr)
if not os.path.isdir(etp_sheet_dr):
    os.makedirs(etp_sheet_dr)
if not os.path.isdir(ndvi_sheet_dr):
    os.makedirs(ndvi_sheet_dr)
eta_sheet_df.to_csv(os.path.join(eta_sheet_dr, 'average ET mm day.csv'), index=False)
etp_sheet_df.to_csv(os.path.join(etp_sheet_dr, 'average ETp mm day.csv'), index=False)
ndvi_sheet_df.to_csv(os.path.join(ndvi_sheet_dr, 'ndvi.csv'), index=False)
kc_dates_df.to_csv(os.path.join(working_dr, 'data', shp_name, current_time, f'season_{season}', 'date_kc.csv'), index=False)

# export charts
save_plot_maxmeanmin(eta_sheet_df, 'avg ET mmday', os.path.join(et_folder, 'ETa'))
save_plot_maxmeanmin(ndvi_sheet_df, 'NDVI', ndvi_folder)
save_plot_et_etp(eta_sheet_df, etp_sheet_df, 'ET ETp mm day', os.path.join(et_folder, 'ETa_ETp'))

# save_shapefile
shapefile_image(project_shp, field_shp, season_dr)

# export priefs
export_priefs(season_dr, et_folder, ndvi_folder)